# **Normalizzazione dei Nomi Aziendali**

## **Introduzione**

Nel processo di deduplicazione e record linkage, è essenziale normalizzare i nomi aziendali per evitare variazioni dovute a abbreviazioni, formati o errori di battitura. In questa fase, implementiamo una strategia per rendere i nomi uniformi e comparabili.

---

## **Step 1: Importazione delle Librerie e del DataFrame**

In [4]:
import re
import pandas as pd
import unidecode

AZIENDE_CSV = '../aziende.csv'
IMPIEGATI_CSV = '../impiegati.csv'

In [5]:
companies_df = pd.read_csv(AZIENDE_CSV)

/tmp/ipykernel_23801/2611636423.py:1: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  companies_df = pd.read_csv(AZIENDE_CSV)


---

## **Step 2: Definizione della Mappa di Sostituzione**
Utilizziamo un dizionario per standardizzare suffissi comuni nei nomi aziendali.
### **Questo va sicuramente migliorato magari passando tutta la prima colonna a chat**

In [7]:
REPLACEMENTS = {
    "&": " and ",
    " inc\.?$": " inc",
    " ltd$": " limited",
    " plc$": " public limited company",
    " co\.?$": " company",
    " corp\.?$": " corp",
    " srl$": " societa a responsabilita limitata",
    " spa$": " societa per azioni",
    " ag$": " aktiengesellschaft",
    " sa$": " societe anonyme",
    " ab$": " aktiebolag",
    " nv$": " naamloze vennootschap",
    " ou$": " osaühing",
    " as$": " aktsiaselts",
    " gmbh$": " gesellschaft mit beschränkter haftung",
    " holdings$": " holding",
    " corporation$": " corp",
    " industries$": " industry",
    " group$": " group",
    " company limited$": " limited",
    " technology$": " tech",
    " solutions$": " solution",
    " systems$": " system",
}

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\.'
<>:7: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\.'
<>:7: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_23801/2264042508.py:3: SyntaxWarning: invalid escape sequence '\.'
  " inc\.?$": " inc",
/tmp/ipykernel_23801/2264042508.py:6: SyntaxWarning: invalid escape sequence '\.'
  " co\.?$": " company",
/tmp/ipykernel_23801/2264042508.py:7: SyntaxWarning: invalid escape sequence '\.'
  " corp\.?$": " corp",


---

## **Step 3: Funzione di Normalizzazione**

Questa funzione applica una serie di trasformazioni per ottenere nomi uniformi:

- Conversione in minuscolo
- Rimozione di accenti
- Eliminazione di caratteri speciali
- Sostituzione dei suffissi tramite `REPLACEMENTS`

In [13]:
def normalize_name(name):
    """Normalizza i nomi delle aziende."""
    if pd.isna(name) or not isinstance(name, str):
        return ""
    
    name = name.lower().strip()  # Minuscolo e rimozione spazi extra
    name = unidecode.unidecode(name)  # Rimuove accenti e caratteri speciali
    name = re.sub(r'[^a-z0-9\s&.,-]', '', name)  # Pulisce caratteri inutili
    name = re.sub(r'\s+', ' ', name)  # Sostituisce spazi multipli con uno solo
    name = re.sub(r'"([^"]+)"','', name) # Rimuove testo tra virgolette
    
    # Applica le sostituzioni dalla mappa REPLACEMENTS
    for pattern, replacement in REPLACEMENTS.items():
        name = re.sub(pattern, replacement, name)
    
    return name.strip()

---

## **Step 4: Applicazione alla Colonna 'company_name'**

Eseguiamo la normalizzazione sulla colonna contenente i nomi aziendali.

In [14]:
companies_df['normalized_name'] = companies_df['company_name'].apply(normalize_name)

---

## **Step 5: Visualizzazione e salvataggio del Risultato**

Mostriamo i primi risultati per verificare la trasformazione
e salva tutto in un csv

In [16]:
display(companies_df[['company_name', 'normalized_name']].head(10))
companies_df[['company_name', 'normalized_name']].to_csv('normalized_company_name.csv', index=False)

,company_name,normalized_name
0,"""all about me"" full stop limited",all about me full stop limited
1,"""q"" (chester green) management company limited",q chester green management limited
2,#sinob,sinob
3,'all aboard' shops limited,all aboard shops limited
4,'q' accountancy services limited,q accountancy services limited
5,'q' accountancy services limited,q accountancy services limited
6,'r' accommodation ltd,r accommodation limited
7,(dia) distribuidora internacional de alimentación,dia distribuidora internacional de alimentacion
8,.net it systemhaus,.net it systemhaus
9,0044 limited,0044 limited


---

## **Conclusioni**

Questo approccio garantisce una normalizzazione efficace dei nomi aziendali, riducendo le variazioni dovute a differenze di scrittura, abbreviazioni e formati. È facilmente estendibile aggiornando la mappa `REPLACEMENTS` con nuove regole di sostituzione.

